In [2]:
import os
import warnings
from datetime import timedelta
warnings.filterwarnings('ignore')
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

drive_location = 'E:'

df_nurse_record = pd.read_csv(f"{drive_location}/new_nursing_0909.csv")
df_nurse_record = df_nurse_record[df_nurse_record['RecordUnit'] == 'ICUC']


unique_numbers_list = set(df_nurse_record['AlsUnitNo'].unique().tolist())
pkl_list = set([int(i.replace('.pkl', '')) for i in os.listdir(f'{drive_location}/pkl/') if i.endswith('.pkl')])

patient_id_list = list(pkl_list & unique_numbers_list)

In [3]:
# v4
import pickle as pkl
import pandas as pd
from datetime import timedelta
from collections import Counter
import re

def label_normalization(text: str = ''):
    text = text.replace('?', '').replace('!', '')
    text = text.replace('>', ' > ').replace('<', ' < ')
    text = re.sub(r'\s{2,}', ' ', text).strip()
    return text

# 기술적 알람 목록 로드
def load_technical_alarms(file_path="Filtered_AlarmLabelList.txt"):
    """기술적 알람 목록을 파일에서 로드하여 정규화"""
    technical_alarms = set()
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        
        for line in lines:
            line = line.strip()
            if not line:  # 빈 줄 건너뛰기
                continue
            
            # 슬래시로 구분된 여러 라벨 처리
            if "/" in line:
                labels = [label.strip() for label in line.split("/")]
            else:
                labels = [line]
            
            # 각 라벨을 정규화하여 저장
            for label in labels:
                if label:  # 빈 문자열이 아닌 경우만
                    normalized_label = label.lower().strip().replace(" ", "")
                    if normalized_label:
                        technical_alarms.add(normalized_label)
        
        print(f"기술적 알람 목록 로드 완료: {len(technical_alarms)}개 라벨")
        
    except Exception as e:
        print(f"기술적 알람 목록 로드 오류: {e}")
    
    return technical_alarms

def normalize_alarm_label(label):
    """알람 라벨을 비교를 위해 정규화"""
    if not label:
        return ""
    return str(label).lower().replace(" ", "").strip()

def is_only_technical_alarms(label_list):
    """Label 리스트가 기술적 알람만 포함하는지 확인"""
    if not label_list or not isinstance(label_list, list):
        return False
    
    # 빈 리스트나 None만 있는 경우
    valid_labels = [label for label in label_list if label and str(label).strip() not in ["None", "[]", ""]]
    if not valid_labels:
        return False
    
    # 모든 라벨이 기술적 알람인지 확인
    for label in valid_labels:
        normalized_label = normalize_alarm_label(label)
        if normalized_label not in TECHNICAL_ALARMS:
            return False  # 하나라도 임상적 알람이 있으면 False
    
    return True  # 모두 기술적 알람인 경우만 True

# 전역 변수로 기술적 알람 목록 로드
TECHNICAL_ALARMS = load_technical_alarms()

def preprocess_data(patient_id):
    pickle = pkl.load(open(f'{drive_location}/pkl/{patient_id}.pkl', 'rb'))
    icuc = pickle[pickle['NURSING_RecordUnit']=='ICUC']

    # 필요한 컬럼 리스트
    required_columns = [
        'AlsUnitNo', 'TimeStamp',
        'Label', 'Severity',
        'ABP_WAVEFORM', 'ECG_WAVEFORM', 'PPG_WAVEFORM', 'RESP_WAVEFORM',
        'ABP_time_diff_sec', 'II_time_diff_sec', 'Pleth_time_diff_sec', 'Resp_time_diff_sec',
        'SpO2_numeric', 'Pulse_numeric', 'ST_numeric', 'Tskin_numeric', 
        'ABP_numeric', 'NBP_numeric', 'HR_numeric', 'RR_numeric',
        'SpO2_numeric_time_diff_sec', 'Pulse_numeric_time_diff_sec', 
        'ST_numeric_time_diff_sec', 'Tskin_numeric_time_diff_sec', 
        'ABP_numeric_time_diff_sec', 'NBP_numeric_time_diff_sec', 
        'HR_numeric_time_diff_sec', 'RR_numeric_time_diff_sec',
    ]
    
    # 모든 필요한 컬럼을 순서대로 생성
    filtered = pd.DataFrame()
    for col in required_columns:
        if col in icuc.columns:
            filtered[col] = icuc[col].copy()
        else:
            filtered[col] = pd.NA
            print(f"Warning: Missing column: {col}")
    
    # Severity 처리
    filtered['Severity'] = filtered['Severity'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
    filtered.insert(
        filtered.columns.get_loc('Severity')+1, 
        'SeverityColor', 
        filtered['Severity'].map({0: "Red", 1:"Yellow", 2:"ShortYellow", 3:"SevereCyan", 4:"Cyan", 5:"SilentCyan", 6:"White"})
    )

    filtered_nursing = df_nurse_record[df_nurse_record['AlsUnitNo'] == patient_id][
        ['TimeStamp', 'AssessmentNm', 'ImplementationNm', 'AttributeNm', 'AttributeDetail', 'AttributeDetailValue', 'AttributeDetailValue2', 'InDateTime', 'OutDateTime']
    ].sort_values('TimeStamp')

    # TimeStamp를 datetime으로 변환
    if filtered['TimeStamp'].dtype == 'object':
        filtered['TimeStamp'] = pd.to_datetime(filtered['TimeStamp'])
    if filtered_nursing['TimeStamp'].dtype == 'object':
        filtered_nursing['TimeStamp'] = pd.to_datetime(filtered_nursing['TimeStamp'])

    # 컬럼명 매핑 (InDateTime, OutDateTime 제외)
    column_mapping = {
        'TimeStamp': '시행일시',
        'AssessmentNm': '간호진단프로토콜(코드명)',
        'ImplementationNm': '간호활동(코드명)',
        'AttributeNm': '간호속성코드(코드명)',
        'AttributeDetail': '간호속성명칭',
        'AttributeDetailValue': '속성',
        'AttributeDetailValue2': '속성Text'
    }

    # 앞 뒤로 둘 다 하나씩은 있어야함
    def get_nursing_records_optimized():
        result = []
        admission_in_list = []
        admission_out_list = []
        in_out_consistency_list = []  # In/Out 시간 일관성 체크 리스트 추가
        
        for idx, row in filtered.iterrows():
            timestamp = row['TimeStamp']
            start_time = timestamp - timedelta(minutes=30)
            end_time = timestamp + timedelta(minutes=30)
            
            # 이전 30분 레코드 확인
            before_mask = (filtered_nursing['TimeStamp'] >= start_time) & (filtered_nursing['TimeStamp'] < timestamp)
            before_records = filtered_nursing[before_mask]
            
            # 이후 30분 레코드 확인
            after_mask = (filtered_nursing['TimeStamp'] > timestamp) & (filtered_nursing['TimeStamp'] <= end_time)
            after_records = filtered_nursing[after_mask]
            
            # 한 쪽이라도 비어있으면 []
            if before_records.empty or after_records.empty:
                result.append([])
                admission_in_list.append('')
                admission_out_list.append('')
                in_out_consistency_list.append(False)  # 데이터가 없으므로 일관성도 False
            else:
                # 전체 범위 레코드 가져오기
                mask = (filtered_nursing['TimeStamp'] >= start_time) & (filtered_nursing['TimeStamp'] <= end_time)
                records = filtered_nursing[mask]
                
                # In/Out 시간 일관성 체크
                unique_in_times = records['InDateTime'].dropna().unique()
                unique_out_times = records['OutDateTime'].dropna().unique()
                
                # 모든 간호기록의 InDateTime과 OutDateTime이 각각 동일한지 확인
                # NaN이 아닌 값들 중에서 unique 값이 1개만 있어야 일관성 있음
                is_consistent = (len(unique_in_times) <= 1) and (len(unique_out_times) <= 1)
                in_out_consistency_list.append(is_consistent)
                
                # 첫 번째 레코드의 InDateTime, OutDateTime 값 (단일 값)
                in_value = records['InDateTime'].iloc[0] if len(records) > 0 and pd.notna(records['InDateTime'].iloc[0]) else ''
                out_value = records['OutDateTime'].iloc[0] if len(records) > 0 and pd.notna(records['OutDateTime'].iloc[0]) else ''
                
                admission_in_list.append(in_value)
                admission_out_list.append(out_value)
                
                # NursingRecords에는 InDateTime, OutDateTime 제외하고 넣기
                records_without_admission = records.drop(columns=['InDateTime', 'OutDateTime'])
                result.append(records_without_admission.rename(columns=column_mapping).to_dict('records'))
        
        return result, admission_in_list, admission_out_list, in_out_consistency_list

    # 함수 호출 및 할당
    nursing_records, admission_ins, admission_outs, in_out_consistency = get_nursing_records_optimized()
    filtered['NursingRecords_ba30'] = nursing_records
    filtered['AdmissionIn'] = admission_ins
    filtered['AdmissionOut'] = admission_outs
    filtered['InOutConsistent'] = in_out_consistency  # 일관성 플래그 추가

    # 필터링 조건 적용 - 존재하는 컬럼만 체크
    waveform_cols = ['ABP_time_diff_sec', 'II_time_diff_sec', 'Pleth_time_diff_sec', 'Resp_time_diff_sec']
    waveform_cols += ['SpO2_numeric_time_diff_sec', 'Pulse_numeric_time_diff_sec', 
        'ST_numeric_time_diff_sec', 'Tskin_numeric_time_diff_sec', 
        'ABP_numeric_time_diff_sec', 'NBP_numeric_time_diff_sec', 
        'HR_numeric_time_diff_sec', 'RR_numeric_time_diff_sec',]
    existing_waveform_cols = [col for col in waveform_cols if col in filtered.columns and filtered[col].notna().any()]
    
    # 조건 생성
    conditions = []
    
    # Waveform 컬럼 조건 (존재하는 컬럼만)
    if existing_waveform_cols:
        conditions.append((filtered[existing_waveform_cols] <= 60).all(axis=1))
    
    # NursingRecords 조건
    conditions.append(filtered['NursingRecords_ba30'].apply(lambda x: len(x) > 0))
    
    # In/Out 시간 일관성 조건 추가
    conditions.append(filtered['InOutConsistent'])
    
    # 모든 조건 적용
    if conditions:
        final_mask = conditions[0]
        for condition in conditions[1:]:
            final_mask = final_mask & condition
        filtered = filtered[final_mask].copy()
    
    # InOutConsistent 컬럼 제거 (최종 데이터에는 필요 없음)
    filtered = filtered.drop(columns=['InOutConsistent'])
    
    # 라벨 특수문자 제거
    filtered['Label'] = filtered['Label'].apply(
        lambda x: [label_normalization(item) for item in x] if isinstance(x, list) else x
    )
    
    # Technical Alarm 인 경우 isView=False로 처리해버리기
    filtered['isView'] = filtered['Label'].apply(
        lambda labels: not is_only_technical_alarms(labels)
    )
    
    filtered['isSelected'] = False # 뷰어에서 T/F 가 선택되었나요? 의 여부
    filtered['Classification'] = False
    filtered['Comment'] = ''

    # Numeric Data 가 모두 정상인 경우
    numeric_cols = [
        'SpO2_numeric', 'Pulse_numeric', 'ST_numeric', 'Tskin_numeric',
        'ABP_numeric', 'NBP_numeric', 'HR_numeric', 'RR_numeric'
    ]
    filtered[numeric_cols] = filtered[numeric_cols].apply(
        pd.to_numeric, errors='coerce'
    )
    filtered = filtered.dropna(subset=numeric_cols, how='any')  # all로 하면 전부 NaN일 때만 제거


    return filtered

기술적 알람 목록 로드 완료: 58개 라벨


In [ ]:
import pickle as pkl
import os

def save_processed_data(patient_id, output_dir=f'{drive_location}/pre_processed1/'):
    filtered = preprocess_data(patient_id)
    
    if len(filtered) > 0:
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        output_path = os.path.join(output_dir, f'{patient_id}_processed.pkl')
        with open(output_path, 'wb') as f:
            pkl.dump(filtered, f)
        print(f"{patient_id}: {len(filtered)} rows saved")
    else:
        print(f"{patient_id}: No rows, skipped")

def batch_process(patient_ids):
    for patient_id in patient_ids:
        try:
            save_processed_data(patient_id)
        except Exception as e:
            print(f"{patient_id}: Error - {e}")


# !rm -rf DATA/
# !mkdir DATA
# !rm -rf sicu_alarms.db
for i in patient_id_list:
    try:
        filtered = preprocess_data(i)
        if len(filtered) > 0:
            with open(f'DATA/{i}.pkl', 'wb') as f:
                pkl.dump(filtered, f)
            print(f"{i}: {len(filtered)} rows saved")
        else:
            print(f"{i}: No rows, skipped")
    except Exception as e:
        print(f"{i}: Error - {e}")




8660993: No rows, skipped
11526147: No rows, skipped
11466763: No rows, skipped
1949715: No rows, skipped
11319320: No rows, skipped
11448345: No rows, skipped
10532891: No rows, skipped
11059230: No rows, skipped
11618341: No rows, skipped
10811448: 25 rows saved
2586684: No rows, skipped
11456575: No rows, skipped
11452483: No rows, skipped
11460687: No rows, skipped
11417682: No rows, skipped
9648217: No rows, skipped
10264681: No rows, skipped
4526187: No rows, skipped
11405419: No rows, skipped
4776045: No rows, skipped
9879666: No rows, skipped
11688056: 84 rows saved
10645625: No rows, skipped
11417722: No rows, skipped
11350145: No rows, skipped
11079810: No rows, skipped
11393161: No rows, skipped
4685968: No rows, skipped
3723418: No rows, skipped
9971876: No rows, skipped
11559076: No rows, skipped
11421863: No rows, skipped
2937002: No rows, skipped
10842283: No rows, skipped
11542704: No rows, skipped
11520179: No rows, skipped
11407540: No rows, skipped
11505853: No rows,

KeyboardInterrupt: 

In [6]:
!python pkl_to_sqlite.py

Creating SQLite database: sicu_alarms.db
Found 57 PKL files to convert

[1/57] Converting 10052298.pkl...
  - Loaded 1 rows, 36 columns
  - Created table with 36 columns
  - Column 'Label' contains array/list data
  - Column 'ABP_WAVEFORM' contains array/list data
  - Column 'ECG_WAVEFORM' contains array/list data
  - Column 'PPG_WAVEFORM' contains array/list data
  - Column 'RESP_WAVEFORM' contains array/list data
  - Column 'NursingRecords_ba30' contains array/list data
  - Successfully inserted 1 rows

[2/57] Converting 10114394.pkl...
  - Loaded 9 rows, 36 columns
  - Created table with 36 columns
  - Column 'Label' contains array/list data
  - Column 'ABP_WAVEFORM' contains array/list data
  - Column 'ECG_WAVEFORM' contains array/list data
  - Column 'PPG_WAVEFORM' contains array/list data
  - Column 'RESP_WAVEFORM' contains array/list data
  - Column 'NursingRecords_ba30' contains array/list data
  - Successfully inserted 9 rows

[3/57] Converting 10138773.pkl...
  - Loaded 2 row